In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from monsterui.all import *
import json

In [2]:
app,rt,contacts,Contact = fast_app('data/contacts.db',hdrs=Theme.blue.headers(),
                                    id=int, first=str, last=str, phone=str, email=str, errors=str,
                                    pk='id', live=True)

In [3]:
if not contacts(): 
    with open('contacts.json','r') as f: data = json.load(f)
    contacts.insert_all(data)

### Table with contacts

In [13]:
def header_render(col): 
    return Th(col.title() if col != 'actions' else '', expand=True)

def action_buttons(contact):
    return DivHStacked(
        Button(UkIcon('eye',cls='mr-2'), 'View', cls=ButtonT.ghost, hx_get=f'/contacts/{contact.id}', hx_target='#modal-container', hx_swap="innerHTML"),
        Button(UkIcon('pencil', cls='mr-2'),"Edit", cls=ButtonT.ghost, hx_get=f'/contacts/{contact.id}/edit')
    )

def contacts_table(q=None):
    rows = filter_contacts(q)
    return Table(
        Thead(
            Tr(*[Th(col) for col in ['First', 'Last', 'Phone', 'Email', 'Actions']])
        ),
        Tbody(
            *[Tr(
                Td(c.first or ''),
                Td(c.last or ''),
                Td(c.phone or ''),
                Td(c.email or ''),
                Td(action_buttons(c))
            ) for c in rows]
        ),
        cls=(TableT.hover, TableT.divider, TableT.responsive),
        id='contacts-table'
    )

def filter_contacts(q=None):
    if not q: return contacts(order_by='id')
    q = q.lower()
    return [o for o in contacts(order_by='id')
            if q in (o.first or '').lower()
            or q in (o.last or '').lower()
            or q in (o.email or '').lower()]

    
page_heading = Div(cls='space-y-2')(
                    H1('Contacts'), P("Manage your contacts!", cls=TextPresets.muted_sm))

In [14]:
def view_contact_modal(contact):
    return Modal(
        ModalBody(
            DivVStacked(
                DivHStacked(Strong("First Name:"), P(contact.first or '-')),
                DivHStacked(Strong("Last Name:"), P(contact.last or '-')),
                DivHStacked(Strong("Phone:"), P(contact.phone or '-')),
                DivHStacked(Strong("Email:"), P(contact.email or '-')),
                cls='space-y-4'
            )
        ),
        header = ModalHeader(H3("Contact Details")),
        footer = ModalCloseButton("Close", cls=ButtonT.primary),
        id = 'contact-modal',
        open=True  # This ensures the modal opens automatically
    )

In [15]:
@rt("/")
def get(): return Redirect('/contacts')


@rt("/contacts/{id:int}")
def get(id:int): return view_contact_modal(contacts[id])

@rt("/contacts")
def get(q:str=None):
    search = Form(
        DivHStacked(
            Input(name='q', value=q, placeholder='Search contacts...',
                cls='w-full md:w-2/3 lg:w-1/2',
                hx_get='/contacts/search',
                hx_trigger='keyup changed delay:500ms',
                hx_target='#contacts-table',
                hx_include='[name="q"]'),
            Button("Search", type='submit')
       ),
       cls='mt-8'
    )
    return Container(page_heading, search, contacts_table(q), Div(id='modal-container'))

@rt("/contacts/search")
def get(q: str = ''): return contacts_table(q)

In [16]:
server.stop()

In [17]:
server = JupyUvi(app)

In [18]:
HTMX()

In [ ]:
# running at http://localhost:8000/